In [1]:
from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
crawled_df = pd.read_csv('crawled_df.csv')

## 1. 과속단속 카메라 개수

In [3]:
overspeed_cam=pd.read_csv('overspeed_cam.csv')
overspeed_cam['geometry'] = list(zip(overspeed_cam['lat'], overspeed_cam['lon']))

In [4]:
def overspeedcam_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## -check

In [5]:
overspeedcam_count(crawled_df['lat'][0], crawled_df['lon'][0], 100, refer_data=overspeed_cam, idx=False)

0

In [6]:
cam_cnt=[]
for i in range(crawled_df.shape[0]):
    cam_cnt.append(overspeedcam_count(crawled_df['lat'][i], crawled_df['lon'][i], 100, overspeed_cam))

In [7]:
print(cam_cnt.count(0))
print(cam_cnt.count(1)) # 총 691개의 자료 중 88개 사고지점에, 카메라 1대가 있었음.

603
88


## 2. 유동인구 평균

In [8]:
floating= pd.read_csv('floating_pop.csv')
floating['geometry'] = list(zip(floating['lat'], floating['lon']))
floating.head()

,lat,lon,mean,geometry
0,37.125623,127.071472,53.367500,"(37.1256226537081, 127.071472079079)"
1,37.125629,127.073161,12.580833,"(37.1256287454303, 127.073160660635)"
2,37.125631,127.073724,12.777500,"(37.1256307706599, 127.073723521266)"
3,37.125633,127.074286,11.696667,"(37.1256327932173, 127.07428638195299)"
4,37.126071,127.070907,51.132500,"(37.1260713170185, 127.070906674399)"


In [9]:
def floating_pop(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon), x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(refer_data['mean'][bool_mask])
  else:
    return sum(refer_data['mean'][bool_mask])

## -check

In [10]:
floating_pop(crawled_df['lat'][0], crawled_df['lon'][0], 100, floating)

1407.046666666667

In [11]:
floating_pop(crawled_df['lat'][0], crawled_df['lon'][0], 100, floating, idx=True)

(Int64Index([8192, 8193, 8194, 8291, 8292, 8293, 8294, 8380, 8381, 8382, 8383,
             8478, 8479],
            dtype='int64'),
 1407.046666666667)

In [12]:
df=floating.iloc[[8192, 8193, 8194, 8291, 8292, 8293, 8294, 8380, 8381, 8382, 8383,
             8478, 8479],:]

In [13]:
center=(crawled_df['lat'][0], crawled_df['lon'][0])
map = folium.Map(location=center, zoom_start=12, width=900, height=700, control_scale=True)

df.apply(lambda row: folium.Marker(location=[row['lat'], row['lon']],
                                                    popup=row['mean']).add_to(map), axis=1)

folium.Marker(center, icon=folium.Icon(color='red')).add_to(map)
# display map
map

# 3. 법규위반

In [14]:
child_ped=pd.read_csv('child_pedestrian.csv')
child_ped_pro=pd.read_csv('child_pedestrian_protect.csv')
child_ped_sig=pd.read_csv('child_pedestrian_signal.csv')

In [15]:
child_ped['geometry']=list(zip(child_ped['lat'],child_ped['lon']))
child_ped_pro['geometry']=list(zip(child_ped_pro['lat'],child_ped_pro['lon']))
child_ped_sig['geometry']=list(zip(child_ped_sig['lat'],child_ped_sig['lon']))

In [16]:
#어린이 보행/자전거 전체 사고 횟수
def child_pedestrian_acc_cnt(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## -check

In [17]:
print(child_pedestrian_acc_cnt(crawled_df['lat'][3], crawled_df['lon'][3], radi=20, refer_data=child_ped))
print(child_pedestrian_acc_cnt(crawled_df['lat'][3], crawled_df['lon'][3], radi=20, refer_data=child_ped_pro))## -check
print(child_pedestrian_acc_cnt(crawled_df['lat'][3], crawled_df['lon'][3], radi=20, refer_data=child_ped_sig))

4
0
3


# 4. 과속방지턱

In [18]:
bump=pd.read_csv('bump.csv')
bump['geometry'] = list(zip(bump['lat'], bump['lon']))
bump.head()

,index,도로명,설치장소명,과속방지턱높이,과속방지턱폭,과속방지턱연장,보차분리여부,연속형여부,lon,lat,geometry
0,0,오산로,소망빌 앞,10,3.6,5.4,N,Y,127.068954,37.130852,"(37.1308521, 127.0689542)"
1,1,동부대로,태영철강재공장 앞,10,3.6,5.0,N,Y,127.076305,37.137946,"(37.1379463, 127.0763048)"
2,2,밀머리로,메디마트 앞,0,3.6,5.4,Y,Y,127.075092,37.141466,"(37.1414665, 127.0750925)"
3,3,오산로,다인하우스 앞,10,3.6,10.0,Y,Y,127.071169,37.143833,"(37.1438328, 127.071169)"
4,4,운천로,운천고등학교 앞,10,3.6,6.0,Y,Y,127.080466,37.155872,"(37.155872, 127.0804662)"


In [19]:
def nearest_bump(lat, lon, radi, refer_data, idx = False):
    dist = refer_data[['index','geometry']].apply(lambda x: (geodesic((lat, lon), x[1]).m, x[0]), axis = 1)
    dist = min(dist, key = lambda x :x[0])
                                        
    if idx:
        return dist
    else:
        return dist[0]

## -check

In [20]:
radi=1
nearest_bump(crawled_df['lat'][0], crawled_df['lon'][0], radi, refer_data=bump, idx=True)

(331.9926124415689, 22)

# 5. 불법주정차

In [21]:
parking=pd.read_csv('parking.csv')
parking['geometry'] = list(zip(parking['lat'], parking['lon']))
parking.head()

,단속일자,단속위치,행정구역,스쿨존여부,lon,lat,단속종류,geometry
0,20181101,금암동 여계산로 부근,경기도 오산시 세교동,N,127.043462,37.170237,이동형CCTV단속,"(37.170237, 127.043462)"
1,20181101,세교동 세마역로19번길,경기도 오산시 세교동,N,127.044770,37.181695,이동형CCTV단속,"(37.181695, 127.04477)"
2,20181101,세교동 수목원로 부근,경기도 오산시 양산동,N,127.042200,37.186357,이동형CCTV단속,"(37.186357, 127.0422)"
3,20181101,오산동 오산로272번길 부,경기도 오산시 원동,N,127.071732,37.150557,이동형CCTV단속,"(37.150557, 127.07173200000001)"
4,20181101,오산동 성호대로93번길,경기도 오산시 부산동,N,127.071757,37.150457,이동형CCTV단속,"(37.150457, 127.071757)"


In [22]:
def parking_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """

  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

## -check

In [23]:
parking_count(crawled_df['lat'][0], crawled_df['lon'][0], 100, refer_data=parking, idx=False)

215